# Benchmarking of prototype


We are going to use the test data and feed it into Redis

In [59]:
import os
import asyncio
import logging
from typing import Optional
from pydantic import BaseModel, Field, validator
from typing import Optional
from datetime import datetime
import time
import httpx


import pyarrow.parquet as pq


In [60]:
class Record(BaseModel):
    input_report: str
    output_report: str
    datetime_est: Optional[str] = None
    close: Optional[float] = None
    signal: Optional[int] = None
    conviction: Optional[float] = None
    causality: Optional[str] = None

    @validator("datetime_est", pre=True, always=True)
    def validate_datetime_format(cls, v):
        if v is None:
            return None
        try:
            # Parse datetime to enforce format
            dt = datetime.strptime(v, "%Y-%m-%d %H:%M:%S")
            return dt.strftime("%Y-%m-%d %H:%M:%S")
        except ValueError:
            raise ValueError("datetime_est must be in 'YYYY-MM-DD HH:MM:SS' Format!")

/var/folders/13/z6c3b3d50nnbcf08brkd87dw0000gn/T/ipykernel_69529/1940412412.py:10: PydanticDeprecatedSince20: Pydantic V1 style `@validator` validators are deprecated. You should migrate to Pydantic V2 style `@field_validator` validators, see the migration guide for more details. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.11/migration/
  @validator("datetime_est", pre=True, always=True)


In [61]:
table = pq.read_table("df_events.parquet")
records = [dict(zip(table.column_names, row)) for row in zip(*table.to_pydict().values())]


## Redis Insert latency benchmark:


Checking Prometheus metrics whe have that using the query 

`histogram_quantile(0.95, rate(redis_insert_duration_seconds_bucket{operation="astore"}[20h]))` 
we can get the p95 latency:

```bash
{instance="trading-api-container:8005", job="trading-platform", operation="astore"}	            0.7463349422954485
```

which is about 0.75 seconds for entries in the full dataset. And given that we embedded + inserted all records in about 5-10s each time, we can say that our program handles multiple inserting request in a decent way!

In [65]:
async def post_record(client, url, record_dict):
    response = await client.post(url, json=record_dict)
    response_json = response.json()
    #print(response_json['response'])

async def main():
    url = "http://localhost:8006/insert_data"
    async with httpx.AsyncClient() as client:
        tasks = []
        for _record in records[:50]: # Only first 50 records for testing
            await asyncio.sleep(0.2)
            record = _record.copy()
            record['datetime_est'] = str(record['datetime_est']) # Enforce string type since in the dataset they are in pd.timestamp type
            final_record = Record(**record)
            tasks.append(post_record(client, url, final_record.model_dump()))
        await asyncio.gather(*tasks)

await main()

21:09:44 httpx INFO   HTTP Request: POST http://localhost:8006/insert_data "HTTP/1.1 200 OK"
21:09:44 httpx INFO   HTTP Request: POST http://localhost:8006/insert_data "HTTP/1.1 200 OK"
21:09:44 httpx INFO   HTTP Request: POST http://localhost:8006/insert_data "HTTP/1.1 200 OK"
21:09:44 httpx INFO   HTTP Request: POST http://localhost:8006/insert_data "HTTP/1.1 200 OK"
21:09:44 httpx INFO   HTTP Request: POST http://localhost:8006/insert_data "HTTP/1.1 200 OK"
21:09:44 httpx INFO   HTTP Request: POST http://localhost:8006/insert_data "HTTP/1.1 200 OK"
21:09:44 httpx INFO   HTTP Request: POST http://localhost:8006/insert_data "HTTP/1.1 200 OK"
21:09:44 httpx INFO   HTTP Request: POST http://localhost:8006/insert_data "HTTP/1.1 200 OK"
21:09:44 httpx INFO   HTTP Request: POST http://localhost:8006/insert_data "HTTP/1.1 200 OK"
21:09:44 httpx INFO   HTTP Request: POST http://localhost:8006/insert_data "HTTP/1.1 200 OK"
21:09:44 httpx INFO   HTTP Request: POST http://localhost:8006/insert_

# Redis semantic caching with the `/decide` endpoint (30 requests)


Now we are going to send concurrent request to check metrics of the caching of prompts against LLM responses

In [66]:
async def post_record(client, url, record_dict):
    response = await client.post(url, json=record_dict, params={"manual_embedding": True})
    response_json = response.json()


# The same way as before, we just change the payload and send the prompt in this schema: {"prompt": "string"}
async def main():
    url = "http://localhost:8006/decide"
    async with httpx.AsyncClient() as client:
        tasks = []
        for _record in records[:30]:
            await asyncio.sleep(0.5)
            record = _record.copy()
            tasks.append(post_record(client, url, {"prompt": record.get("input_report")}))
        await asyncio.gather(*tasks)

await main()

21:10:11 httpx INFO   HTTP Request: POST http://localhost:8006/decide?manual_embedding=true "HTTP/1.1 200 OK"
21:10:11 httpx INFO   HTTP Request: POST http://localhost:8006/decide?manual_embedding=true "HTTP/1.1 200 OK"
21:10:11 httpx INFO   HTTP Request: POST http://localhost:8006/decide?manual_embedding=true "HTTP/1.1 200 OK"
21:10:11 httpx INFO   HTTP Request: POST http://localhost:8006/decide?manual_embedding=true "HTTP/1.1 200 OK"
21:10:11 httpx INFO   HTTP Request: POST http://localhost:8006/decide?manual_embedding=true "HTTP/1.1 200 OK"
21:10:11 httpx INFO   HTTP Request: POST http://localhost:8006/decide?manual_embedding=true "HTTP/1.1 200 OK"
21:10:11 httpx INFO   HTTP Request: POST http://localhost:8006/decide?manual_embedding=true "HTTP/1.1 200 OK"
21:10:11 httpx INFO   HTTP Request: POST http://localhost:8006/decide?manual_embedding=true "HTTP/1.1 200 OK"
21:10:11 httpx INFO   HTTP Request: POST http://localhost:8006/decide?manual_embedding=true "HTTP/1.1 200 OK"
21:10:11 h

## After inserting 50 recods and performing a semantic cache with 30 records we have the following benchmarks:



### Average embedding duration: 

`openai_embedding_duration_seconds_sum / openai_embedding_duration_seconds_count`

```Python
{instance="trading-api-container:8005", job="trading-platform", operation="embedding"}  	0.47188677787780764
```

### p0.95 latency for embeddings: 

`histogram_quantile(0.95,openai_embedding_duration_seconds_bucket)`

```Python
{instance="trading-api-container:8005", job="trading-platform", operation="embedding"}	    0.7187499999999999
```

### Average latency for semantic cache match: 

`semantic_cache_duration_seconds_sum / semantic_cache_duration_seconds_count`

```Python
{instance="trading-api-container:8005", job="trading-platform", operation="acheck"}	        0.004074819882710775
```
### p0.95 latency for semantic cache: 

`histogram_quantile(0.95, semantic_cache_duration_seconds_bucket)` 

```Python
{instance="trading-api-container:8005", job="trading-platform", operation="acheck"}	        0.009375

```

## Now lets dive into the actual benchmarks of the endpoint:

### Average latency: 

`http_request_duration_seconds_sum{endpoint="/decide"} / http_request_duration_seconds_count{endpoint="/decide"}`

```Python
{endpoint="/decide", instance="trading-api-container:8005", job="trading-platform", method="POST"}	    0.4941580216089884

```

### p0.95 Latency: 

`histogram_quantile(0.95, http_request_duration_seconds_bucket{endpoint="/decide"})`

```Python
{endpoint="/decide", instance="trading-api-container:8005", job="trading-platform", method="POST"}	0.7249999999999999
```


# *Core bottleneck is embedding time for full prompts!*


# Then, how can we improve this embedding speed? 

There are two solutions aproaches:

1. Using a higher speed embedder: We can use simpler or smaller embedding models, that comes at risks of losing embedding quality that might capture important information. Keep in mind that I already swiched Open AI's embedding model from large to small, which reduced p0.95 latency of the full /decide request by around 0.25s (Using test dataset), which is a significant improvement on the context of what we are trying to achieve.

2. Pre-embed all heavy/static prompts (instruction prompts, prior analyzed news, etc), keep them in a traditional cache to store their vector representation so each time we require their embedding we can perform an extremely fast look up. At runtime, whenever an incoming news chunk arrives to the system, we quickly embed it using the model of our choice, then, try recent news + prompts combinations and compose their vectors via sum+norm to approximate the combined prompt+context embedding for cache lookup. Then we can perform a semantic cache lookup in order to attempt to retrieve a real trading decision if the similarity for one of the composed vectors have a similarity > than a predefined threshhold τ.

3. Mixed aproach; We can try the first two approaches at the same time, by trying different embedding models!



We could do the following:

 - Have all prompts + chunks of analized news in a regular cache, lets call it Embedding cache (EC). Refer to https://redis.io/docs/latest/develop/ai/redisvl/user_guide/embeddings_cache/

 - Have in the semantic cache (sc) the sum of those chunks and prompts normalized together with the response containing the signal and metadata

 - When a new chunk of news comes in (which is significatively smaller than a full prompt), we quickly embed it and add it into our pre-embedded news + prompts cache.

 - Then we try to append the incoming news into different pre-embedded chunks + news combinations. We approximate the full-prompt vecor embedding using a vector compositions of every good candidate for a hit in the SC.

- I could not integrate this system described above due to tight deadlines.
